In [ ]:
import cv2
import mediapipe as mp

# Initialize mediapipe hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,  # <-- Changed to 2
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.7)

mp_draw = mp.solutions.drawing_utils

# Open the webcam
cap = cv2.VideoCapture(0)

# Tip landmarks of each finger
finger_tips = [4, 8, 12, 16, 20]

# Check if a finger is up
def is_finger_up(hand_landmarks, finger_index, hand_label):
    if finger_index == 0:  # Thumb
        if hand_label == "Right":
            return hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x
        else:  # Left hand
            return hand_landmarks.landmark[4].x > hand_landmarks.landmark[3].x
    else:
        return hand_landmarks.landmark[finger_tips[finger_index]].y < hand_landmarks.landmark[finger_tips[finger_index] - 2].y

# Get handedness (Right/Left)
def get_hand_label(results, hand_index):
    if results.multi_handedness:
        return results.multi_handedness[hand_index].classification[0].label
    return "Unknown"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip for natural interaction
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(rgb_frame)
    finger_count = 0

    if result.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            hand_label = get_hand_label(result, i)  # "Left" or "Right"

            for j in range(5):
                if is_finger_up(hand_landmarks, j, hand_label):
                    finger_count += 1

    # Display finger count
    cv2.putText(frame, f'Fingers: {finger_count}', (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    cv2.imshow("10-Finger Hand Tracker", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
